In [3]:
# Imports
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# Pytorch Gpu Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

import matplotlib.pyplot as plt

# Matplotlib svg plots for better pictures
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

import seaborn as sns
import pandas as pd
from torchsummary import summary

In [4]:
# Import datasets
iris_data = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
)
print(iris_data.describe())
data = torch.tensor(iris_data[iris_data.columns[0:4]].values).float()

# Transfer species to number
labels = torch.zeros(len(data), dtype=torch.long)
labels[iris_data.species == "versicolor"] = 1
labels[iris_data.species == "virginica"] = 2

       sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000


In [5]:
# Create class for the model


class NNiris(nn.Module):
    def __init__(self, nUnits, nLayers):
        super().__init__()

        # Create dictionary to store the layers
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers

        ### Input Layer
        self.layers["input"] = nn.Linear(4, nUnits)

        # Hidden Layers
        for i in range(nLayers):
            self.layers[f"hidden{i}"] = nn.Linear(nUnits, nUnits)

        ### Output Layers
        self.layers["output"] = nn.Linear(nUnits, 3)

    # Forward Pass
    def forward(self, x):
        # Input Layer
        x = F.relu(self.layers["input"](x))

        # Hidden Layers
        print(self.nLayers)
        for i in range(self.nLayers):
            print(f"{i} th forward")
            x = F.relu(self.layers[f"hidden{i}"](x))

        # Output Layers
        x = self.layers["output"](x)

        return x

In [6]:
# Model Instance
nUnitsPerLayer = 12
nLayers = 5
nn_net = NNiris(nUnitsPerLayer, nLayers)
summary(nn_net)

Layer (type:depth-idx)                   Param #
├─ModuleDict: 1-1                        --
|    └─Linear: 2-1                       60
|    └─Linear: 2-2                       156
|    └─Linear: 2-3                       156
|    └─Linear: 2-4                       156
|    └─Linear: 2-5                       156
|    └─Linear: 2-6                       156
|    └─Linear: 2-7                       39
Total params: 879
Trainable params: 879
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─ModuleDict: 1-1                        --
|    └─Linear: 2-1                       60
|    └─Linear: 2-2                       156
|    └─Linear: 2-3                       156
|    └─Linear: 2-4                       156
|    └─Linear: 2-5                       156
|    └─Linear: 2-6                       156
|    └─Linear: 2-7                       39
Total params: 879
Trainable params: 879
Non-trainable params: 0

In [7]:
# Sanity testing the model
# 20 Samples 4 dimensions
tmp_x = torch.randn(20, 4)
print(tmp_x.shape)
print(" ")

y = nn_net(tmp_x)

print(" ")
print(y.shape)
print(" ")
print(y)

torch.Size([20, 4])
 
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
 
torch.Size([20, 3])
 
tensor([[-0.0592, -0.1397, -0.0228],
        [-0.0577, -0.1392, -0.0230],
        [-0.0590, -0.1393, -0.0228],
        [-0.0600, -0.1396, -0.0226],
        [-0.0578, -0.1396, -0.0229],
        [-0.0593, -0.1395, -0.0227],
        [-0.0584, -0.1393, -0.0229],
        [-0.0589, -0.1397, -0.0228],
        [-0.0556, -0.1398, -0.0232],
        [-0.0566, -0.1386, -0.0228],
        [-0.0558, -0.1394, -0.0230],
        [-0.0582, -0.1402, -0.0229],
        [-0.0595, -0.1397, -0.0228],
        [-0.0565, -0.1396, -0.0230],
        [-0.0578, -0.1408, -0.0229],
        [-0.0562, -0.1412, -0.0233],
        [-0.0604, -0.1398, -0.0227],
        [-0.0576, -0.1394, -0.0229],
        [-0.0600, -0.1394, -0.0226],
        [-0.0580, -0.1388, -0.0229]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [10]:
# Function to train the model
def trainTheModel(model, n_epochs, lr, data, labels):

    # Loss function and optimizer
    lossfun = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # Loop over epochs
    for epoch in range(n_epochs):

        # Forward pass
        yHat = model(data)

        # Compute Loss
        loss = lossfun(yHat, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Final Forward pass and accuracy calculation
    predictions = model(data)
    predlabels = torch.argmax(predictions, axis=1)
    acc = 100 * torch.mean((predlabels == labels).float())

    # Total trainable params in the model
    nParams = sum([p.numel() for p in model.parameters() if p.requires_grad])

    return acc, nParams

In [14]:
n_epochs = 2500
lr = 0.01

acc, nParams = trainTheModel(nn_net, n_epochs, lr, data, labels)

acc.squeeze().cpu().detach()

5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th forward
5
0 th forward
1 th forward
2 th forward
3 th forward
4 th for

tensor(98.6667)